In [1]:
import os
import cv2
import pickle
from tqdm import tqdm
import pandas as pd 
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
print(tf.__version__)

1.15.0


In [2]:
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
pickle_in = open("./data/train_x.pickle","rb")
train_x = pickle.load(pickle_in)

pickle_in = open("./data/train_y.pickle","rb")
train_y = pickle.load(pickle_in)

pickle_in = open("./data/test_x.pickle","rb")
test_x = pickle.load(pickle_in)

pickle_in = open("./data/test_y.pickle","rb")
test_y = pickle.load(pickle_in)

print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(2850, 60, 60, 3)
(2850, 3)
(150, 60, 60, 3)
(150, 3)


In [4]:
num_classes = 3
n_input = 10800
learning_rate = 0.001
training_iters = 10
batch_size = 512
display_step = 20

In [5]:
img_size=60
num_channels=3
x = tf.compat.v1.placeholder(tf.float32, shape=[None, img_size,img_size,num_channels]) 
y_ = tf.compat.v1.placeholder(tf.float32, [None, num_classes])
print('Shape of placeholder',x.shape, y_.shape)

Shape of placeholder (?, 60, 60, 3) (?, 3)


In [6]:
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def maxpool2d(x, k=2):
    return tf.nn.max_pool2d(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

In [7]:
weights = {
    'w1': tf.Variable(tf.random.normal([5, 5, 3, 32]),name='w1'),
    'w2': tf.Variable(tf.random.normal([5, 5, 32, 64]),name='w2'),
    'w3': tf.Variable(tf.random.normal([5, 5, 64, 128]),name='w3'),
    'wd1': tf.Variable(tf.random.normal([8 * 8 * 128, 2048]),name='wd1'),  
    'wout': tf.Variable(tf.random.normal([2048, num_classes]),name='wout')
}

biases = {
    'b1': tf.Variable(tf.random.normal([32]),name='b1'),
    'b2': tf.Variable(tf.random.normal([64]),name='b2'),
    'b3': tf.Variable(tf.random.normal([128]),name='b3'),
    'bd1': tf.Variable(tf.random.normal([2048]),name='bd1'),
    'bout': tf.Variable(tf.random.normal([num_classes]),name='bout')
}

In [8]:
def conv_net(x, weights, biases):
        
    # reshape input to 60x60x3 size
    x = tf.reshape(x, shape=[-1, 60, 60, 3])  
    
    print("----------------------------------------------------------------------------")
    print("size of x is")
    print(x.shape)
    
  
    conv1 = conv2d(x, weights['w1'], biases['b1'])
    conv1 = maxpool2d(conv1, k=2)
    print("----------------------------------------------------------------------------")
    print("size after 1st conv layer is ")
    print(conv1.shape)

    
    #input is 30*30*32 image
    # Convolution Layer
    conv2 = conv2d(conv1, weights['w2'], biases['b2'])
    conv2 = maxpool2d(conv2, k=2)
    print("----------------------------------------------------------------------------")
    print("size after 2nd conv and pooling layer is")
    print(conv2.shape)
    
    
    ### third conv layer
    # input is 15*15*64 image
    # Convolution Layer
    conv3 = conv2d(conv2, weights['w3'], biases['b3'])
  
    conv3 = maxpool2d(conv3, k=2)
    print("----------------------------------------------------------------------------")
    print("size after 3rd conv and pooling layer is")
    print(conv3.shape)
    
    
    
    #input is 8*8*128 
    # Fully connected layer
    # Reshape conv3 output to fit fully connected layer input   = 8*8*128 = 8192
    fc1 = tf.reshape(conv3, [-1, weights['wd1'].get_shape().as_list()[0]])
    print("----------------------------------------------------------------------------")
    print("shape after flattening the image")
    print(fc1)  #8192 is the output
    
    
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    print("----------------------------------------------------------------------------")
    print("shape after fully connected layer")
    print(fc1)
    
    
    # Output, class prediction
    # finally we multiply the fully connected layer with the weights and add a bias term. 
    out = tf.add(tf.matmul(fc1, weights['wout']), biases['bout'])
    print("----------------------------------------------------------------------------")
    print("Output layer")
    return out

In [9]:
model = conv_net(x, weights, biases)
print(model)

----------------------------------------------------------------------------
size of x is
(?, 60, 60, 3)
----------------------------------------------------------------------------
size after 1st conv layer is 
(?, 30, 30, 32)
----------------------------------------------------------------------------
size after 2nd conv and pooling layer is
(?, 15, 15, 64)
----------------------------------------------------------------------------
size after 3rd conv and pooling layer is
(?, 8, 8, 128)
----------------------------------------------------------------------------
shape after flattening the image
Tensor("Reshape_1:0", shape=(?, 8192), dtype=float32)
----------------------------------------------------------------------------
shape after fully connected layer
Tensor("Relu_3:0", shape=(?, 2048), dtype=float32)
----------------------------------------------------------------------------
Output layer
Tensor("Add_1:0", shape=(?, 3), dtype=float32)


In [10]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=y_))
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [11]:
init = tf.compat.v1.global_variables_initializer()

In [12]:
cost_history=[]
n_epochs =500

sess = tf.compat.v1.Session()
sess.run(init)
train_y=train_y.todense()
for i in range(n_epochs):
    a, c = sess.run([optimizer, cost], feed_dict={x: train_x, y_: train_y})  
    cost_history = np.append(cost_history,c)  
    print('epoch : ', i, ' - ', 'cost: ', c)

epoch :  0  -  cost:  4312453.0
epoch :  1  -  cost:  4400386.0
epoch :  2  -  cost:  5290323.0
epoch :  3  -  cost:  4702528.0
epoch :  4  -  cost:  2932652.0
epoch :  5  -  cost:  2252504.2
epoch :  6  -  cost:  1554134.9
epoch :  7  -  cost:  2601072.2
epoch :  8  -  cost:  2489491.5
epoch :  9  -  cost:  1365592.6
epoch :  10  -  cost:  971081.94
epoch :  11  -  cost:  1462924.2
epoch :  12  -  cost:  1688363.6
epoch :  13  -  cost:  1668195.8
epoch :  14  -  cost:  1409376.2
epoch :  15  -  cost:  958993.5
epoch :  16  -  cost:  765100.4
epoch :  17  -  cost:  1012586.2
epoch :  18  -  cost:  1113777.4
epoch :  19  -  cost:  937414.25
epoch :  20  -  cost:  872413.94
epoch :  21  -  cost:  866078.9
epoch :  22  -  cost:  722928.7
epoch :  23  -  cost:  658729.5
epoch :  24  -  cost:  772838.5
epoch :  25  -  cost:  825607.4
epoch :  26  -  cost:  711903.5
epoch :  27  -  cost:  565382.1
epoch :  28  -  cost:  563027.5
epoch :  29  -  cost:  657916.06
epoch :  30  -  cost:  659782.

epoch :  247  -  cost:  7073.1797
epoch :  248  -  cost:  21906.027
epoch :  249  -  cost:  4186.034
epoch :  250  -  cost:  8707.62
epoch :  251  -  cost:  8050.029
epoch :  252  -  cost:  4333.7637
epoch :  253  -  cost:  7505.701
epoch :  254  -  cost:  4708.512
epoch :  255  -  cost:  3882.905
epoch :  256  -  cost:  5511.8066
epoch :  257  -  cost:  4238.807
epoch :  258  -  cost:  3300.0757
epoch :  259  -  cost:  3822.5771
epoch :  260  -  cost:  4339.092
epoch :  261  -  cost:  3487.0452
epoch :  262  -  cost:  3136.2834
epoch :  263  -  cost:  3364.602
epoch :  264  -  cost:  3028.36
epoch :  265  -  cost:  2446.9958
epoch :  266  -  cost:  2487.4277
epoch :  267  -  cost:  2827.9358
epoch :  268  -  cost:  2365.759
epoch :  269  -  cost:  2109.4966
epoch :  270  -  cost:  2117.5515
epoch :  271  -  cost:  2092.5522
epoch :  272  -  cost:  1925.0621
epoch :  273  -  cost:  1938.1713
epoch :  274  -  cost:  1997.9612
epoch :  275  -  cost:  1774.6401
epoch :  276  -  cost:  162

In [13]:
import matplotlib.pyplot as plt
plt.figure()
plt.plot(range(0,500), cost_history[0:500])
plt.ylabel('COST')
plt.xlabel('EPOCH')
plt.ylim([0, 1000])
plt.show()

In [14]:
test_y=test_y.todense() 
correct_prediction = tf.equal(tf.argmax(model,1), tf.argmax(y_,1))   
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("Accuracy: ", sess.run(accuracy, feed_dict={x: test_x, y_:test_y}))

Accuracy:  0.64666665
